In [32]:
import pandas as pd

In [33]:
apartments = pd.read_csv('../data/curated/geometry_property.csv')
tram_stops = pd.read_csv('../data/curated/tram_stop.csv')
train_stations = pd.read_csv('../data/curated/train_station.csv')

In [40]:
apartments.loc[apartments['Unnamed: 0'] == 1855]

,Unnamed: 0,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),...,Apartments - total (no.),Median weekly household rental payment ($),Rented (no.),Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_stops,num_stations,total_transport
1855,1855,"207B/58 Myrtle St, \nIVANHOE VIC 3079",IVANHOE,550.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,1529.0,440.0,1505.0,16.0,343.0,NaN,NaN,0.0,0.0,0.0


In [35]:
train_stations

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95230710206772 -37.781179806470384)
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93932910356588 -37.78812681562748)
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.9361719034498 -37.79425383545)
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.9425759409124 -37.80740580596864)
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542285377925 -37.78864386946561)
...,...,...,...,...,...,...,...
215,19965,Coburg Railway Station (Coburg),-37.742345,144.963336,1,Upfield,POINT (144.9633418243668 -37.742331846983326)
216,19966,Moreland Railway Station (Coburg),-37.754485,144.961823,1,Upfield,POINT (144.96182910331285 -37.75447185695532)
217,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield,POINT (144.96069006277764 -37.761228876919425)
218,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield,POINT (144.9595927822653 -37.767707796884636)


In [36]:
# Calculate Haversine Distance
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

In [37]:
for index, row1 in apartments.iterrows():
    lon1 = row1['longitude']
    lat1 = row1['latitude']
    
    num_stops = 0 
    for index_stops, row2 in tram_stops.iterrows():
        lon2 = row2['LONGITUDE']
        lat2 = row2['LATITUDE']

        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stops += 1

    apartments.at[index, 'num_stops'] = num_stops

In [38]:
for index2, row3 in apartments.iterrows():
    lon1 = row3['longitude']
    lat1 = row3['latitude']
    
    num_stations = 0
    for index_stations, row4 in train_stations.iterrows():
        lon2 = row4['LONGITUDE']
        lat2 = row4['LATITUDE']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.6:
            num_stations += 1
    apartments.at[index2, 'num_stations'] = num_stations

In [39]:
apartments['total_transport'] = apartments['num_stops'] + apartments['num_stations']
sorted_apartments = apartments.sort_values(by='total_transport', ascending=False)
sorted_apartments

,Unnamed: 0,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),...,Apartments - total (no.),Median weekly household rental payment ($),Rented (no.),Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_stops,num_stations,total_transport
1137,1137,"7/25 Gladstone Avenue, \nARMADALE VIC 3143",ARMADALE,510.0,9436.0,4321.5,38.0,6445.0,0.0,0.0,...,3223.0,421.0,1921.0,27.0,438.0,-37.863649,145.023319,25.0,1.0,26.0
2426,2426,"17/1 Duke Street, \nST KILDA VIC 3182",ST KILDA,430.0,30189.0,12631.5,72.8,23753.0,0.0,0.0,...,14987.0,776.0,7769.0,576.0,1852.0,-37.867441,144.989594,22.0,1.0,23.0
17,17,"218/60 Siddeley Street, \nDOCKLANDS VIC 3008",DOCKLANDS,620.0,15942.0,6522.9,32.5,13715.0,0.0,0.0,...,10602.0,411.0,5056.0,18.0,1602.0,-37.822193,144.953748,22.0,1.0,23.0
1033,1033,"713/60 Siddeley Street, \nDOCKLANDS VIC 3008",DOCKLANDS,950.0,15942.0,6522.9,32.5,13715.0,0.0,0.0,...,10602.0,411.0,5056.0,18.0,1602.0,-37.822193,144.953748,22.0,1.0,23.0
2827,2827,"1012/60 Siddeley Street, \nDOCKLANDS VIC 3008",DOCKLANDS,580.0,15942.0,6522.9,32.5,13715.0,0.0,0.0,...,10602.0,411.0,5056.0,18.0,1602.0,-37.822193,144.953748,22.0,1.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,1853,"10/262 Poath Road, \nHUGHESDALE VIC 3166",HUGHESDALE,550.0,7644.0,3803.9,37.9,5330.0,0.0,0.0,...,547.0,416.0,1019.0,56.0,344.0,-37.903283,145.075147,0.0,0.0,0.0
1855,1855,"207B/58 Myrtle St, \nIVANHOE VIC 3079",IVANHOE,550.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,1529.0,440.0,1505.0,16.0,343.0,NaN,NaN,0.0,0.0,0.0
1856,1856,"112/1031 Heidelberg Road, \nIVANHOE VIC 3079",IVANHOE,550.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,1529.0,440.0,1505.0,16.0,343.0,NaN,NaN,0.0,0.0,0.0
1857,1857,"G16/1-5 Westley Avenue, \nIVANHOE VIC 3079",IVANHOE,550.0,12561.0,2487.4,40.9,8357.0,0.0,0.0,...,1529.0,440.0,1505.0,16.0,343.0,NaN,NaN,0.0,0.0,0.0
